Vaguely following the tutorial: 
https://www.tensorflow.org/text/tutorials/text_classification_rnn

In [18]:
from curses.ascii import isalpha, isdigit
from plistlib import InvalidFileException
import sys
import os
import numpy as np
from scipy import spatial
from Bill import Bill

In [19]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [20]:
def make_bills(folder:str):
    bill_locations = os.listdir(folder)
    bill_locations = list(map(lambda loc:folder + '/' + loc, bill_locations))
    bills = []
    for bill_file in bill_locations:
        if bill_file[-3:] == 'txt':
            try:
                bills.append(Bill(bill_file))
            except InvalidFileException as e:
                continue
    return bills

In [21]:
GloVe_file = '/Users/lucasgover/Downloads/glove.6B/glove.6B.50d.txt'

In [22]:
emmbed_dict = {}
with open(GloVe_file,'r') as f:
  for line in f:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    emmbed_dict[word]=vector

In [23]:
len(list(list(emmbed_dict.values())[0]))

50

In [25]:
print(emmbed_dict['penguin'])
print(emmbed_dict['penguins'])

[ 0.26809   0.19536  -0.85239   0.17078   0.49344   0.17885  -1.1485
 -1.5259    0.32724  -0.41434   0.32616   1.0239    0.55419   0.3874
  1.0458    0.31049   0.067686 -0.14538  -1.2247    0.8357    0.067415
  0.45555   0.41626   0.30002  -0.2766    0.28402  -0.78093  -0.36034
 -0.6686   -0.40467   0.33729  -0.39184   0.1391    1.0417   -0.89186
 -0.28363  -0.63136  -1.0775    0.18568  -0.98521   0.85771  -0.25934
  0.25918   0.64003  -0.27659   0.45614   0.56947  -0.36858  -0.84004
 -0.26615 ]
[ 0.24527    0.62001   -0.24089    0.07491    0.0024258  0.53981
 -1.6029     0.21598   -1.3212    -0.098482   0.73315    0.7586
  0.19109    1.0154     0.74009    0.068095   0.67282   -0.95646
 -2.1396     0.69536   -1.3298    -1.0708     0.081902  -0.43348
 -0.58345   -0.24443    0.30174   -0.040186   0.46538   -1.1714
  0.386      1.5198     0.091624   0.068236  -0.73884    0.21032
 -0.36036    0.22627    0.27641   -0.32114   -0.17974   -0.11887
  0.38149    0.33808   -0.16036    0.46927   -

In [26]:
# Import Bills Here
BillsLocation = '/Users/lucasgover/Desktop/AI431_Projects/AIFinal/Bills_2021-2022'
bills = make_bills(BillsLocation)


#Turn into tf datasets


In [34]:
def get_words(bill:Bill):
    words = []
    current_word = ''
    for character in bill.text.lower():
        if isdigit(character) or isalpha(character):
            current_word += character
        else:
            if len(current_word) > 0:
                words.append(current_word)
                current_word = ''

    return words

In [35]:
def dataset_to_embedding_dataset(words:list):
    embeddings = []
    for word in words:
        if(word in emmbed_dict):
            emmbeddings += emmbed_dict[word]
        else:
            # Do something else?
            print(word + " was not in the embedding dict!")
    return embeddings

In [36]:
def bills_to_dataset(bills_list:list):
    CHUNK_SIZE = 100
    texts = []
    labels = []
    for bill in bills_list:
        words = get_words(bill)
        num_chunks = len(words) // CHUNK_SIZE
        chunks = []
        while(len(words) > CHUNK_SIZE):
            chunks += [dataset_to_embedding_dataset(words[:CHUNK_SIZE])]
            words = words[CHUNK_SIZE:]
        texts += chunks
        if bill.party == 'D' or 'd':
            labels += [-1] * num_chunks
        elif bill.party == 'R' or 'r':
            labels += [1] * num_chunks
        elif bills.party == 'I' or 'i':
            labels += [-1] * num_chunks
        else:
            labels += [0] * num_chunks
    return texts,labels
        

In [37]:
bill_texts,bill_labels = bills_to_dataset(bills)

TypeError: argument of type 'builtin_function_or_method' is not iterable

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

In [ ]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [ ]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [ ]:
print([layer.supports_masking for layer in model.layers])

In [ ]:
# predict on a sample text without padding.

sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

In [ ]:
# predict on a sample text with padding

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

The two cells above should output the same thing? I'm not sure what padding is right now. 

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=30)

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
predictions = model.predict(np.array([sample_text]))